In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from archiv.utils import create_bestand

In [ ]:
file = 'legacy_data/gtrans_publikationen_Wienbibliothek_Tagblattarchiv.xlsx'
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')

In [ ]:
df = pd.read_excel(file).fillna('False')

In [ ]:
for i, row in df.iterrows():
    if row['Bibliographische Angaben']:
        signatur = row['Bibliographische Angaben'][:500]
    if row['Teil von Bestand ']:
        bestand = create_bestand(row['Teil von Bestand '])[-1]
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
        item.location = bestand
        item.save()
        if row['Titel'] != 'False':
            item.title = row['Titel']
        if row['Zusammenfassung'] != 'False':
            item.abstract = row['Zusammenfassung']
        if row['Kommentar'] != 'False':
            item.notes = row['Kommentar']
        if row['Permalink'] != 'False':
            item.permalink = row['Permalink']
        if row['Datum original '] != 'False':
            try:
                item.written_date = row['Datum original '].replace('00:00:00', '')
            except Exception as e:
                print(e, row['Datum original '])
        if row['Datum normalisiert DD/MM/YYYY)'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Datum normalisiert DD/MM/YYYY)'])
            except Exception as e:
                print(e)
        if row['Typ'] != 'False':
            type_vocab, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(row['Typ'])
            )
            type_vocab.scheme.add(res_type_scheme)
            item.res_type = type_vocab
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Institutionen'] != 'False':
            for x in row['Institutionen'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Personen'] != 'False':
            for x in row['Personen'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
        if row['ErzeugerIn des Dokuments'] != 'False':
            for x in row['ErzeugerIn des Dokuments'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.creator_person.add(pl)
        item.save()